# หมายเหตุ
เป็นการทดลองเพื่อเปรียบเทียบประสิทธิภาพของ Embedding model

*   โดยจะใช้ โมเดล intfloat/multilingual-e5-large-instruc

*   จะเน้นค่า Recall@ k   เท่านั้น



คำเเนะนำ
  ให้ทำการเชื่อม google drive ก่อนทุกครั้ง


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


โหลดสิ่งที่จำเป็น

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [4]:
# ติดตั้ง PyTorch เวอร์ชันล่าสุด
!pip install torch==2.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# 🚀 ลบ PyTorch และ Transformers เก่าทั้งหมด
!pip uninstall -y torch torchvision torchaudio transformers

# 🚀 ติดตั้ง PyTorch ที่รองรับ CUDA ของ Colab (CUDA 11.8)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 🚀 ติดตั้ง Transformers เวอร์ชันที่รองรับ
!pip install transformers --upgrade

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:0

In [5]:
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.5 MB/s eta 0:

ทำการ embed คำถามของ Evaluation datasest ด้วย intfloat/multilingual-e5-large-instruct

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
import numpy as np

# 🚀 โหลดโมเดลและ Tokenizer (Hugging Face)
model_name = "intfloat/multilingual-e5-large-instruct"  # ใช้โมเดลเดียวกับที่เก็บใน Vector Database
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16)  # ใช้ Half-Precision เพื่อประหยัดหน่วยความจำ
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # ตั้งเป็นโหมดประเมินผลเพื่อประหยัดหน่วยความจำ

# 🚀 ฟังก์ชันสร้าง Embedding ของคำถาม
def embed_text_batch(texts):
    inputs = tokenizer(
        texts,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

# 🚀 โหลด Evaluation Dataset
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 สร้าง Embeddings สำหรับคำถามทั้งหมด (Batch)
batch_size = 10  # แบ่งเป็น batch ละ 10 คำถาม
all_embeddings = []

for i in range(0, len(evaluation_data), batch_size):
    batch_questions = [item['question'] for item in evaluation_data[i:i + batch_size]]

    batch_embeddings = embed_text_batch(batch_questions)
    all_embeddings.extend(batch_embeddings.tolist())

    print(f"✅ batch ที่ {i // batch_size + 1} embed แล้ว")

    # 🚀 เคลียร์ Cache ของ GPU เพื่อประหยัดหน่วยความจำ
    if device == torch.device("cuda"):
        torch.cuda.empty_cache()

# 🚀 เพิ่ม Embeddings กลับเข้า Evaluation Dataset
for i, item in enumerate(evaluation_data):
    item['embedding'] = all_embeddings[i]

print("✅ สร้าง Embeddings สำหรับคำถามใน Evaluation Dataset สำเร็จ")

# 🚀 กำหนดเส้นทางที่ต้องการบันทึกไฟล์ (Google Drive)
output_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2.json'

# 🚀 บันทึก Evaluation Dataset พร้อม Embeddings ลงไฟล์ JSON
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(evaluation_data, f, ensure_ascii=False, indent=4)

print(f"✅ บันทึก Evaluation Dataset พร้อม Embeddings ที่: {output_file}")


เชื่อม vector database

In [ ]:
# 🚀 2. เชื่อมต่อกับ Chroma Database ที่มีอยู่แล้ว
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database'  # ระบุ path ของฐานข้อมูล
client = chromadb.PersistentClient(path=db_path)

# โหลด Collection ที่มีอยู่แล้ว
collection = client.get_collection(name="recommendations")
print("✅ เชื่อมต่อกับ Chroma DB สำเร็จ")


เช็คว่าสามารถดึงข้อมูลได้มั้ย

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG ดึงคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(question_embedding, top_k=3):
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )
    if results['documents']:
        return results['documents'][0]
    return []

print("✅ พร้อมสำหรับการค้นหาคำแนะนำด้วย RAG")

# 🚀 ทดลองดึงคำแนะนำที่ใกล้เคียงสำหรับคำถามแรก
test_question = evaluation_data[0]['question']
test_embedding = evaluation_data[0]['embedding']

recommendations = retrieve_recommendations(test_embedding)
print(f"\n🔎 คำถาม: {test_question}\n✅ คำแนะนำที่ใกล้เคียง:\n")

for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec}")


ให้ค่า K = 3

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=3):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=3)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item['category'],
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_K=3.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [7]:
import json

# โหลดไฟล์ evaluation dataset (คำถาม-คำตอบจริง)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# โหลดไฟล์ผลลัพธ์ RAG (คำถาม + คำแนะนำที่ดึงมา)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_K=3.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 3  # จำนวนคำแนะนำที่ดึงมา (top_k)

def precision_at_k(relevant, retrieved, k):
    # นับจำนวนคำที่ตรงกันระหว่าง relevant กับ retrieved (จำกัดที่ k ตัวแรก)
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / k

def recall_at_k(relevant, retrieved, k):
    # นับคำที่ตรงกันใน top_k / จำนวนคำที่เกี่ยวข้องจริงทั้งหมด
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / len(relevant) if relevant else 0

def mean_reciprocal_rank(relevant, retrieved):
    # หาลำดับแรกที่คำตอบจริงปรากฏใน retrieved
    for idx, rec in enumerate(retrieved, 1):
        if rec in relevant:
            return 1 / idx
    return 0

# เก็บผลลัพธ์คำนวณต่างๆ
precisions = []
recalls = []
mrrs = []

for item in rag_results:
    # คำตอบจริง (ในรูปแบบ list เพื่อรองรับหลายคำตอบในอนาคต)
    relevant_answers = [item['answer']]

    # คำแนะนำที่ดึงมา (list of strings)
    retrieved = item['recommendations']

    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

# คำนวณค่าเฉลี่ยของ Precision, Recall, MRR ทั้งหมด
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)
avg_mrr = sum(mrrs) / len(mrrs)

print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


Precision@3: 0.139
Recall@3: 0.416
Mean Reciprocal Rank (MRR): 0.345


ให้ค่า K = 5

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=5):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=5)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item['category'],
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_K=5.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [8]:
import json

# โหลดไฟล์ evaluation dataset (คำถาม-คำตอบจริง)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# โหลดไฟล์ผลลัพธ์ RAG (คำถาม + คำแนะนำที่ดึงมา)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_K=5.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 5  # จำนวนคำแนะนำที่ดึงมา (top_k)

def precision_at_k(relevant, retrieved, k):
    # นับจำนวนคำที่ตรงกันระหว่าง relevant กับ retrieved (จำกัดที่ k ตัวแรก)
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / k

def recall_at_k(relevant, retrieved, k):
    # นับคำที่ตรงกันใน top_k / จำนวนคำที่เกี่ยวข้องจริงทั้งหมด
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / len(relevant) if relevant else 0

def mean_reciprocal_rank(relevant, retrieved):
    # หาลำดับแรกที่คำตอบจริงปรากฏใน retrieved
    for idx, rec in enumerate(retrieved, 1):
        if rec in relevant:
            return 1 / idx
    return 0

# เก็บผลลัพธ์คำนวณต่างๆ
precisions = []
recalls = []
mrrs = []

for item in rag_results:
    # คำตอบจริง (ในรูปแบบ list เพื่อรองรับหลายคำตอบในอนาคต)
    relevant_answers = [item['answer']]

    # คำแนะนำที่ดึงมา (list of strings)
    retrieved = item['recommendations']

    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

# คำนวณค่าเฉลี่ยของ Precision, Recall, MRR ทั้งหมด
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)
avg_mrr = sum(mrrs) / len(mrrs)

print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


Precision@5: 0.096
Recall@5: 0.480
Mean Reciprocal Rank (MRR): 0.360


ให้ค่า K = 10

In [ ]:
import json
import chromadb

# 🚀 โหลด Evaluation Dataset พร้อม Embeddings
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_dataset_with_embeddings_v2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

print(f"✅ โหลด Evaluation Dataset พร้อม Embeddings จำนวนคำถาม-คำตอบ: {len(evaluation_data)}")

# 🚀 เชื่อมต่อกับ Chroma DB
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database'
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name="recommendations")

# 🚀 ฟังก์ชัน RAG (Retrieve) เพื่อค้นหาคำแนะนำที่ใกล้เคียง
def retrieve_recommendations(query_embedding, top_k=10):
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0] if results['documents'] else []

# 🚀 ใช้ RAG ค้นหาคำแนะนำสำหรับทุกคำถามใน Dataset
rag_results = []

print("🔄 กำลังดึงคำแนะนำที่ใกล้เคียงด้วย RAG...")

for idx, item in enumerate(evaluation_data, 1):
    query_embedding = item['embedding']
    recommendations = retrieve_recommendations(query_embedding, top_k=10)

    rag_results.append({
        "question": item['question'],
        "answer": item['answer'],
        "category": item['category'],
        "recommendations": recommendations
    })

    if idx % 100 == 0 or idx == len(evaluation_data):
        print(f"✅ ประมวลผลแล้ว {idx}/{len(evaluation_data)} คำถาม")

# 🚀 บันทึกผลลัพธ์ RAG
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_K=10.json'
with open(rag_results_file, 'w', encoding='utf-8') as f:
    json.dump(rag_results, f, ensure_ascii=False, indent=4)

print(f'✅ บันทึกผลลัพธ์ RAG ที่: {rag_results_file}')


หาค่า Recall

In [9]:
import json

# โหลดไฟล์ evaluation dataset (คำถาม-คำตอบจริง)
evaluation_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'
with open(evaluation_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# โหลดไฟล์ผลลัพธ์ RAG (คำถาม + คำแนะนำที่ดึงมา)
rag_results_file = '/content/drive/MyDrive/LockLearn/evaluation_rag_results_K=10.json'
with open(rag_results_file, 'r', encoding='utf-8') as f:
    rag_results = json.load(f)

K = 10  # จำนวนคำแนะนำที่ดึงมา (top_k)

def precision_at_k(relevant, retrieved, k):
    # นับจำนวนคำที่ตรงกันระหว่าง relevant กับ retrieved (จำกัดที่ k ตัวแรก)
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / k

def recall_at_k(relevant, retrieved, k):
    # นับคำที่ตรงกันใน top_k / จำนวนคำที่เกี่ยวข้องจริงทั้งหมด
    retrieved_k = retrieved[:k]
    hits = sum(1 for r in retrieved_k if r in relevant)
    return hits / len(relevant) if relevant else 0

def mean_reciprocal_rank(relevant, retrieved):
    # หาลำดับแรกที่คำตอบจริงปรากฏใน retrieved
    for idx, rec in enumerate(retrieved, 1):
        if rec in relevant:
            return 1 / idx
    return 0

# เก็บผลลัพธ์คำนวณต่างๆ
precisions = []
recalls = []
mrrs = []

for item in rag_results:
    # คำตอบจริง (ในรูปแบบ list เพื่อรองรับหลายคำตอบในอนาคต)
    relevant_answers = [item['answer']]

    # คำแนะนำที่ดึงมา (list of strings)
    retrieved = item['recommendations']

    p = precision_at_k(relevant_answers, retrieved, K)
    r = recall_at_k(relevant_answers, retrieved, K)
    mrr = mean_reciprocal_rank(relevant_answers, retrieved)

    precisions.append(p)
    recalls.append(r)
    mrrs.append(mrr)

# คำนวณค่าเฉลี่ยของ Precision, Recall, MRR ทั้งหมด
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)
avg_mrr = sum(mrrs) / len(mrrs)

print(f"Precision@{K}: {avg_precision:.3f}")
print(f"Recall@{K}: {avg_recall:.3f}")
print(f"Mean Reciprocal Rank (MRR): {avg_mrr:.3f}")


Precision@10: 0.055
Recall@10: 0.550
Mean Reciprocal Rank (MRR): 0.369
